In [1]:
from __future__ import absolute_import
from __future__ import print_function

# Train

In [7]:
def get_eicu_feature(min_time,skip_time):
    
    from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
    import numpy as np
    import re
    import os
    import csv
    import sys
    import pandas as pd
    import numpy as np
    import random
    from scipy import interp
    import matplotlib.pyplot as plt

    from itertools import cycle
    from sklearn.metrics import roc_curve, auc
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split, StratifiedKFold

    from sklearn.metrics import classification_report,confusion_matrix,f1_score,auc,roc_curve,precision_recall_curve
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc,confusion_matrix, average_precision_score, matthews_corrcoef
    from numpy.random import seed
    from inspect import signature

    import keras
    from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
    import warnings
    warnings.filterwarnings('ignore')

    from sklearn.linear_model import LogisticRegression
    from numpy.random import seed


    import torch.nn as nn
    import optuna
    from torch.utils.data.sampler import WeightedRandomSampler
    import torch
    from torch.utils.data import DataLoader,Dataset
    import torch 
    import torch.nn as nn
    from torch.nn.utils.rnn import pack_sequence

    SEED_VALUE = 36
    seed(SEED_VALUE)
    np.random.seed(SEED_VALUE)
    random.seed(SEED_VALUE)
    os.environ['PYTHONHASHSEED']=str(SEED_VALUE)



    torch.manual_seed(SEED_VALUE)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(SEED_VALUE)
    
    CV = True
    BATCH_SIZE = 128
    TASK = '%d_%d'%(min_time,skip_time)
    
    class BiRNN(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, num_classes, drp):
            super(BiRNN, self).__init__()
            self.num_layers  = num_layers
            self.num_classes = num_classes
            self.drop = nn.Dropout(p = drp)
            self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True)
            self.fc = nn.Linear(hidden_size*2, num_classes) # 2 for bidirection
            self.sigmoid = torch.nn.Sigmoid()
            self.embeds = nn.Embedding(50,5)
            self.hidden_size= hidden_size


        def forward(self, x_num, x_cat , seq_lengths):
            embed = self.embeds(x_cat).reshape((x_cat.size(0), x_cat.size(1), -1))
            x = torch.cat((x_num,embed),-1)
            out, _ =  self.lstm(x) # tensor of shape (batch_size, seq_length, hidden_size*2)
            lstm_out = out[:,-1,:]
            output = self.fc(lstm_out)#torch.cat((lstm_out_fw, lstm_out_bw), -1))
            return output
    
###########################################################
    def plot_cum_auc(x_y, models, savename='plot.png', x_label='False Positive Rate', y_label='True Positive Rate',auprc=False, thr=0):
    
        colors = ['b', 'r','g']
        for i, (x_y_tuple, m) in enumerate(zip(x_y, models)):
            plt.plot(x_y_tuple[0], x_y_tuple[1],label='{}: {:.4f}'.format(m,auc(x_y_tuple[0], x_y_tuple[1])), color=colors[i])
        if auprc:
            plt.plot([0, 1], [thr, thr], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(thr), alpha=.8)

        else:
            plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(0.5000), alpha=.8)

        plt.xlabel(x_label, fontsize=15)
        plt.ylabel(y_label, fontsize=15)
        plt.axhline(0, color='black')
        plt.axvline(0, color='black')

        if auprc:
            legend = plt.legend(loc="upper left", prop={'size': 10}, bbox_to_anchor=(0.6, 0.95))
        else:
            legend = plt.legend(loc="lower right", prop={'size': 10}, bbox_to_anchor=(1, 0.05))
            
        plt.savefig(savename,
                     dpi=400, facecolor='white', transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')
        plt.show()
        
        
    def compute_metrics(X_test,y_test,y_probs):

        fpr, tpr, thresholds = roc_curve(y_test, y_probs)
        specat90 = 1-fpr[tpr>=0.90][0]
        intrp = interp(np.linspace(0, 1, 100), fpr, tpr)
        intrp[0] = 0.0

        roc_auc = auc(fpr, tpr)

        TN,FP,FN,TP = confusion_matrix(y_test, y_probs.round()).ravel()
        PPV = TP/(TP+FP)
        NPV = TN/(TN+FN)


        mcc = matthews_corrcoef(y_test, y_probs.round())
        
        average_precision = average_precision_score(y_test, y_probs)
        precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
        prs = interp(np.linspace(0, 1, 100), recall[::-1], precision[::-1])
        prs[0] = 1.0
        prs[-1] = 0.0

        auc_prc = auc(recall, precision)

        return {'specat90': specat90, 'intrp': intrp,
                'fpr': fpr,
                'tpr': tpr, 'auc': roc_auc,
                'ppv': PPV, 'npv': NPV,
                'auc_prc': auc_prc,
                'prc':precision,
                'prs':prs,
                'rec':recall,
                'mcc': mcc}


    def avg_metrics(results):
    
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0, 1, 100)

        mean_tpr = np.mean([results[k]['intrp'] for k in results], axis=0)
        mean_tpr[-1] = 1.0
        std_tpr = np.std([results[k]['intrp'] for k in results], axis=0)
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std([results[k]['auc'] for k in results])
        ppvs = np.mean([results[k]['ppv'] for k in results])
        npvs = np.mean([results[k]['npv'] for k in results])
        mccs = np.mean([results[k]['mcc'] for k in results])
        specat90 = np.mean([results[k]['specat90'] for k in results])

        mean_precision = np.mean([results[k]['prs'] for k in results], axis=0)
        mean_precision[-1] = 0.0
        #########################
        mean_precision[0] = 1.0
        ########################
        mean_auc_prc = auc(mean_recall, mean_precision)
        std_auc_prc = np.std([results[k]['auc_prc'] for k in results], axis=0)

        print("Mean AUC: {0:0.4f} +- STD{1:0.4f}".format(mean_auc,std_auc))
        print("PPV: {0:0.4f}".format(ppvs))
        print("NPV: {0:0.4f}".format(npvs))
        print("MCC: {0:0.4f}".format(mccs))
        print("Spec@90: {0:0.4f}".format(specat90))

        return {'mean_auc': mean_auc,
                'tpr': mean_tpr,
                'std_auc':std_auc,
                'std_tpr': std_tpr,
                'ppv':ppvs,
                'npv':npvs,
                'mean_auc_prc':mean_auc_prc,
                'std_auc_prc':std_auc_prc,
                'mean_prc':mean_precision,
                'mean_recall':mean_recall,
                'mcc':mccs,
                'spec@90':specat90}

    def plot_auc(result):
        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='red',
                 label='Random', alpha=.8)

        mean_fpr = np.linspace(0,1,100)
        mean_tpr = result['tpr']
        mean_auc = result['mean_auc']
        std_auc = result['std_auc']
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.4f $\pm$ %0.4f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = result['std_tpr']
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()
        
###########################################################################
    
    #pos
    def pos_selection(df_pos,skip_time,min_time):
        posl = []
        posdf = pd.DataFrame(columns=df_pos.columns)
        all_matches = df_pos[df_pos['labelpt'] == df_pos['labelrec']]

        for i, p_id in enumerate(all_matches['patientunitstayid'].unique()):
            df_p_id = df_pos[df_pos['patientunitstayid'] == p_id]
            idx = all_matches[all_matches['patientunitstayid'] == p_id].index[0]
            t = df_pos.iloc[idx].itemoffset
            if t > (min_time + skip_time):
                posl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) & (df_p_id['itemoffset'] > (t-(skip_time+min_time)))])

        posdf = pd.concat(posl,axis=0)
        return posdf

    #neg
    def neg_selection(df_neg,skip_time,min_time):
        negl = []
        negdf = pd.DataFrame(columns=df_neg.columns)

        for i, p_id in enumerate(df_neg['patientunitstayid'].unique()):
            df_p_id = df_neg[df_neg['patientunitstayid'] == p_id]
            t = df_p_id.iloc[-1].itemoffset
            if t > (min_time + skip_time):
                negl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) &(df_p_id['itemoffset'] > (t-(min_time+skip_time)))])
        negdf = pd.concat(negl,axis=0)
        return negdf
    eicu_df  = pd.read_csv("eicu_df_all_24los_normed.csv")

 # ADD VASO
    eicu_df['vaso_dose'] = np.nan
    eicu_df['vaso_dose'] = eicu_df['rate_epinephrine']+eicu_df['rate_norepinephrine'] + eicu_df['rate_phenylephrine']/10 + eicu_df['rate_dopamine']/2
    eicu_df.drop(columns=['rate_epinephrine', 'rate_norepinephrine','rate_phenylephrine','rate_dopamine'],inplace=True)

    
    
    eicu_pos = eicu_df[eicu_df['CAM']==1]
    eicu_neg = eicu_df[eicu_df['CAM']==0]
    
    eicu_pos_filtered = pos_selection(eicu_pos,skip_time,min_time)
    eicu_neg_filtered = neg_selection(eicu_neg,skip_time,min_time)
    eicu_df_filtered = pd.concat([eicu_pos_filtered, eicu_neg_filtered],axis=0)


    trg = eicu_df_filtered.groupby('patientunitstayid')


    idtr = []
    train_np = []
    for idx, frame in trg:
        idtr.append(idx)
        train_np.append(frame)


    columns_ord = ['patientunitstayid', 'itemoffset', 'gender','sofa', 'sofa_wo_gcs', 'age', 'admissionheight',
           'admissionweight', 'Heart Rate', 'O2 Saturation', 'glucose',
           'Temperature (C)', 'sodium', 'BUN', 'WBC x 1000', 'Hemoglobin',
           'Platelets', 'Potassium', 'Chloride', 'Bicarbonate', 'Creatinine',
            'vent_flag', 'vaso_dose', 'CAM']

    def reader_deli(df_list,verbose=1):
        X_noncat = []
        X_cat = []
        deli = []
        nrows = []
        ts = []
        PID = []
        nb_unit_stays = len(df_list)
        for i, df in enumerate(df_list):
            if verbose:
                sys.stdout.write('\rFeed StayID {0} of {1}...'.format(i+1, nb_unit_stays))
            dft = df
            dummy = pd.DataFrame(columns=columns_ord)
            for c in columns_ord:
                dummy[c] = dft[c]        
            dft = dummy
            narr = np.array(dft)
            pid = narr[0,0]
            x_cat    = narr[:,2:5]
            x_noncat = narr[:, 5:-1]
            labeldeli = narr[0, -1]
            time = narr[:,1]
            X_cat.append(x_cat)
            X_noncat.append(x_noncat)
            deli.append(labeldeli)
            ts.append(time)
            nrows.append(narr.shape[0])
            PID.append(pid)
        PID = np.array(PID)    
        X_cat = np.array(X_cat)
        X_noncat = np.array(X_noncat)
        deli = np.array(deli)
        ts= np.array(ts)
        return PID,X_cat,X_noncat,ts,nrows,deli
    
    PID_tr,X_cat_tr,X_num_tr,ts_tr,nrows_tr,y_tr = reader_deli(train_np)
    
    def pad(arr, max_len= min_time):
        tmp = np.zeros((max_len, arr.shape[1]))
        tmp[:arr.shape[0], :arr.shape[1]] = arr
        return tmp  
    def ret_numpy(X):
        X_list = []
        for n in X:
            X_list.append(pad(n))
        return np.array(X_list)
    
    X_cat_tr = ret_numpy(X_cat_tr)
    X_num_tr = ret_numpy(X_num_tr)

    
    X_train = np.concatenate([X_num_tr,X_cat_tr],axis=-1)
    y_train = y_tr

    
    def data_reader(X,y):
        return X,y

    
    class overdata(Dataset):
        def __init__(self, data,label):
            self.data  = torch.FloatTensor(data.astype('float'))
            self.label = torch.FloatTensor(label.astype('float'))

        def __len__(self):
            return len(self.data)

        def __getitem__(self, index):
            label  = self.label[index]
            data   = self.data[index]
            return data,label

    device = "cuda" if torch.cuda.is_available() else "cpu"
    kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}

    def sampler_fn(data, label):
        train_class_sample_count = torch.tensor([(torch.tensor(label) == t).sum() for t in torch.unique(torch.tensor(label), sorted=True)])
        train_weight = 1 / train_class_sample_count.float()
        train_samples_weight = torch.tensor([train_weight[i] for i in torch.tensor(label).long()])
        train_sampler = WeightedRandomSampler(train_samples_weight, len(train_samples_weight))
        return train_sampler
    
    def get_class_weights(data, label):
        neg = data[label==0].shape[0]
        pos = len(data) - neg
        weight_for_0 = (1 / neg)*((pos+neg))/2.0 
        weight_for_1 = (1 / pos)*((pos+neg))/2.0
        return weight_for_0 ,(weight_for_1)
    
    if CV:
        X_train,y_train= data_reader(X_train,y_train)
        
    sequence_length = min_time
    input_size = 33 
        
    def run_train(data_loader,params,data,label):
    
        train_losses, val_losses = [], []

        cw_0,cw_1 = get_class_weights(data,label)
        model = BiRNN(input_size, params['hidden_units'], params['num_layers'], num_classes=params['n_classes'],drp = params['dropout']).to(device)

        # Loss and optimizer
        criterion = nn.CrossEntropyLoss(weight=torch.Tensor([cw_0,cw_1]).to(device))
        optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'],weight_decay=1e-6)

        for epoch in range(params['epochs']):

            true_labels = []
            pred_labels = []

            running_loss = 0

            for data, labels in data_loader:
#                 import pdb;pdb.set_trace()
                model.train()
                data = torch.FloatTensor(data).to(device)
                labels = torch.FloatTensor(labels).to(device)
                labels = labels.long()

                ##
                nonzeros = ~data.cpu().numpy().any(axis=2)
                nonzero_index = np.argmax(nonzeros,axis=1)        
                nonzero_index = torch.LongTensor(nonzero_index)
    #             import pdb;pdb.set_trace()
                nonzero_index = torch.where(nonzero_index==0,torch.LongTensor([12]),nonzero_index)
                seq_lengths, perm_idx = nonzero_index.sort(0, descending=True)

                data = data[perm_idx]
                labels = labels[perm_idx]
                ###
                data = data.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(data[:, :, :18], data[:, : ,18:].long(), seq_lengths)
                loss = criterion(outputs, labels)

                # Backward and optimize
                loss.backward()
                optimizer.step()        
                running_loss += loss.item()
            print("Epoch: {}/{} ".format(epoch+1, params['epochs']),"Training Loss: {:.3f} ".format(running_loss/len(data_loader)))
        
        return model

    params = {'lr':0.000075,'num_layers':1,'hidden_units':128,'n_classes':2,
              'dropout':0.3,'epochs':50}
    
    def run_test(data_loader,model):
        with torch.no_grad():
            correct = 0
            total = 0
            model.eval()
            true_labels, pred_labels = [], []

            for data,labels in data_loader:
                labels = labels.to(device)
                labels = labels.long()
                ###
                nonzeros = ~data.cpu().numpy().any(axis=2)
                data = torch.Tensor(data).to(device)
                nonzero_index = np.argmax(nonzeros,axis=1)        
                nonzero_index = torch.LongTensor(nonzero_index)
                nonzero_index = torch.where(nonzero_index==0,torch.LongTensor([12]),nonzero_index)

                seq_lengths, perm_idx = nonzero_index.sort(0, descending=True)
                data = data[perm_idx]
                labels = labels[perm_idx]
                ###
                outputs = nn.Softmax()(model(data[:, :, :18], data[:, : ,18:].long(), seq_lengths))
                pred = outputs.detach().cpu().numpy()
                pred = pred[:,1]
                true_labels.append(labels)
                pred_labels.append(pred)
            true_labels = [l.item() for labels in true_labels for l in labels]
            pred_labels = [p for labels in pred_labels for p in labels]
            pred_labels = np.array(pred_labels)

            return true_labels,pred_labels

    def model_cv(X,y):
        i = 1
        cv_scores = {}
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED_VALUE)
        for train, test in kfold.split(X, y):
            print(f'Fold: {i}')
            X_tr = X[train]
            X_ts = X[test]
            y_tr = y[train]
            y_ts = y[test]

            data_sampler = sampler_fn(X_tr,y_tr)
            train_data = overdata(X_tr,y_tr)
            test_data  = overdata(X_ts,y_ts)
            train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=False,sampler=data_sampler, **kwargs)
            test_loader  = DataLoader(test_data, batch_size=BATCH_SIZE,shuffle=False, **kwargs)

            model = run_train(train_loader, params, X_tr, y_tr)
            savepath = "eicu_%d_%d_fold_%d_cv_torch.pt"%(min_time,skip_time,i)
            
            
            
            torch.save(model.state_dict(), savepath)
            print(f"Model:{savepath} saved.")

            y_ts,probs = run_test(test_loader,model)
            cv_scores[i] = compute_metrics(X_ts,y_ts,probs)
            i += 1        
        cum_scores = avg_metrics(cv_scores)
        return cum_scores
    


    
    models = ['LSTM']
    results = {}
    for m in models:
        print(f'**********Model: {m} *********')
        results[m] = model_cv(X_train,y_train)
        mean_fpr = np.linspace(0,1,100)
        fpr_tprs = [(mean_fpr, results[m]['tpr']) for m in models]
        plot_cum_auc(fpr_tprs, models, savename='{}_cv_eicu_feat.png'.format(TASK),auprc=False)

   

In [8]:
for m in [12]:
    for s in [48]:
        get_eicu_feature(m,s)

# Attribution

# Data reader

In [ ]:
import pandas as pd
import numpy as np
import sys
import torch


min_time = 24
skip_time = 96


device = "cuda" if torch.cuda.is_available() else "cpu"
kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}


#pos
def pos_selection(df_pos,skip_time,min_time):
    posl = []
    posdf = pd.DataFrame(columns=df_pos.columns)
    all_matches = df_pos[df_pos['labelpt'] == df_pos['labelrec']]

    for i, p_id in enumerate(all_matches['patientunitstayid'].unique()):
        df_p_id = df_pos[df_pos['patientunitstayid'] == p_id]
        idx = all_matches[all_matches['patientunitstayid'] == p_id].index[0]
        t = df_pos.iloc[idx].itemoffset
        if t > (min_time + skip_time):
            posl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) & (df_p_id['itemoffset'] > (t-(skip_time+min_time)))])

    posdf = pd.concat(posl,axis=0)
    return posdf

#neg
def neg_selection(df_neg,skip_time,min_time):
    negl = []
    negdf = pd.DataFrame(columns=df_neg.columns)

    for i, p_id in enumerate(df_neg['patientunitstayid'].unique()):
        df_p_id = df_neg[df_neg['patientunitstayid'] == p_id]
        t = df_p_id.iloc[-1].itemoffset
        if t > (min_time + skip_time):
            negl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) &(df_p_id['itemoffset'] > (t-(min_time+skip_time)))])
    negdf = pd.concat(negl,axis=0)
    return negdf
eicu_df  = pd.read_csv("eicu_df_all_24los_normed.csv")
# mimic_df = pd.read_csv("mimic_df_all_24los_normed.csv")

# ADD VASO
eicu_df['vaso_dose'] = np.nan
eicu_df['vaso_dose'] = eicu_df['rate_epinephrine']+eicu_df['rate_norepinephrine'] + eicu_df['rate_phenylephrine']/10 + eicu_df['rate_dopamine']/2
eicu_df.drop(columns=['rate_epinephrine', 'rate_norepinephrine','rate_phenylephrine','rate_dopamine'],inplace=True)

#     import pdb;pdb.set_trace()

eicu_pos = eicu_df[eicu_df['CAM']==1]
eicu_neg = eicu_df[eicu_df['CAM']==0]

eicu_pos_filtered = pos_selection(eicu_pos,skip_time,min_time)
eicu_neg_filtered = neg_selection(eicu_neg,skip_time,min_time)
eicu_df_filtered = pd.concat([eicu_pos_filtered, eicu_neg_filtered],axis=0)

trg = eicu_df_filtered.groupby('patientunitstayid')
idtr = []
train_np = []
for idx, frame in trg:
    idtr.append(idx)
    train_np.append(frame)


columns_ord = ['patientunitstayid', 'itemoffset', 'gender','sofa', 'sofa_wo_gcs', 'age', 'admissionheight',
       'admissionweight', 'Heart Rate', 'O2 Saturation', 'glucose',
       'Temperature (C)', 'sodium', 'BUN', 'WBC x 1000', 'Hemoglobin',
       'Platelets', 'Potassium', 'Chloride', 'Bicarbonate', 'Creatinine',
        'vent_flag', 'vaso_dose', 'CAM']

def reader_deli(df_list,verbose=1):
    X_noncat = []
    X_cat = []
    deli = []
    nrows = []
    ts = []
    PID = []
    nb_unit_stays = len(df_list)
    for i, df in enumerate(df_list):
        if verbose:
            sys.stdout.write('\rFeed StayID {0} of {1}...'.format(i+1, nb_unit_stays))
        dft = df
        dummy = pd.DataFrame(columns=columns_ord)
        for c in columns_ord:
            dummy[c] = dft[c]        
        dft = dummy
        narr = np.array(dft)
        pid = narr[0,0]
        x_cat    = narr[:,2:5]
        x_noncat = narr[:, 5:-1]
        labeldeli = narr[0, -1]
        time = narr[:,1]
        X_cat.append(x_cat)
        X_noncat.append(x_noncat)
        deli.append(labeldeli)
        ts.append(time)
        nrows.append(narr.shape[0])
        PID.append(pid)
    PID = np.array(PID)    
    X_cat = np.array(X_cat)
    X_noncat = np.array(X_noncat)
    deli = np.array(deli)
    ts= np.array(ts)
    return PID,X_cat,X_noncat,ts,nrows,deli

PID_tr,X_cat_tr,X_num_tr,ts_tr,nrows_tr,y_tr = reader_deli(train_np)

def pad(arr, max_len= min_time):
    tmp = np.zeros((max_len, arr.shape[1]))
    tmp[:arr.shape[0], :arr.shape[1]] = arr
    return tmp  
def ret_numpy(X):
    X_list = []
    for n in X:
        X_list.append(pad(n))
    return np.array(X_list)

X_cat_tr = ret_numpy(X_cat_tr)
X_num_tr = ret_numpy(X_num_tr)

X_train = np.concatenate([X_num_tr,X_cat_tr],axis=-1)
y_train = y_tr


In [11]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
import re
import os
import csv
import sys
import pandas as pd
import numpy as np
import random
from scipy import interp
import matplotlib.pyplot as plt

from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.metrics import classification_report,confusion_matrix,f1_score,auc,roc_curve,precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc,confusion_matrix, average_precision_score, matthews_corrcoef
from numpy.random import seed
from inspect import signature

import keras
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from numpy.random import seed


import torch.nn as nn
from torch.utils.data.sampler import WeightedRandomSampler
import torch
from torch.utils.data import DataLoader,Dataset
import torch 
import torch.nn as nn
from torch.nn.utils.rnn import pack_sequence


from captum.attr import IntegratedGradients,NoiseTunnel
from captum.attr import LayerConductance
from captum.attr import NeuronConductance
from captum.attr import TokenReferenceBase,remove_interpretable_embedding_layer, configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
from scipy import stats
from torch.distributions.normal import Normal

SEED_VALUE = 36
seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
random.seed(SEED_VALUE)
os.environ['PYTHONHASHSEED']=str(SEED_VALUE)



torch.manual_seed(SEED_VALUE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED_VALUE)

In [12]:
feature_name = ['Age', 'Height', 'Weight', 'Heart Rate', 'O2 Saturation', 'glucose',
       'Temperature', 'Sodium', 'BUN', 'WBC', 'Hemoglobin',
       'Platelets', 'Potassium', 'Chloride', 'Bicarbonate', 'Creatinine',
        'Ventilation', 'Vasopressor dose', 'Gender','Sofa', 'Sofa_w/o_gcs']

from torch.utils.data import DataLoader,Dataset

def get_baseline(X,y):
    baseline_num = torch.zeros(X.shape[0],min_time,18)
    baseline_cat = torch.zeros(X.shape[0],min_time,3)
    m = Normal(torch.tensor([0.0]), torch.tensor([1.0]))
    baseline_num_gaus = m.sample((X.shape[0],min_time,18)).squeeze()  
    baseline_num_unif = torch.FloatTensor(X.shape[0],min_time,18).uniform_(0, 1)
    baseline_num = (baseline_num_gaus + baseline_num_unif)/2
    x_baseline = torch.cat((baseline_num,baseline_cat),-1)
    y_baseline = torch.ones(X.shape[0])
    x_baseline = x_baseline.cpu().numpy()
    y_baseline = y_baseline.cpu().numpy()
    
    return x_baseline, y_baseline

class overdata(Dataset):
    def __init__(self, data,label):
        self.data  = torch.FloatTensor(data.astype('float'))
        self.label = torch.FloatTensor(label.astype('float'))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        label  = self.label[index]
        data   = self.data[index]
        return data,label
    
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, drp):
        super(BiRNN, self).__init__()
        self.num_layers  = num_layers
        self.num_classes = num_classes
        self.drop = nn.Dropout(p = drp)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True)
        self.fc = nn.Linear(hidden_size*2, num_classes) # 2 for bidirection
        self.sigmoid = torch.nn.Sigmoid()
        self.embeds = nn.Embedding(50,5)
        self.hidden_size= hidden_size


    def forward(self, x_num, x_cat , seq_lengths):
        embed = self.embeds(x_cat).reshape((x_cat.size(0), x_cat.size(1), -1))
        x = torch.cat((x_num,embed),-1)
        out, _ =  self.lstm(x) # tensor of shape (batch_size, seq_length, hidden_size*2)
        lstm_out = out[:,-1,:]
        output = self.fc(lstm_out)#torch.cat((lstm_out_fw, lstm_out_bw), -1))
        return output

In [ ]:
from shap import summary_plot
from captum.attr import ShapleyValueSampling,IntegratedGradients,GuidedBackprop

alg = ["GB","IG","SVS"]


def get_feature_ranking_for_shap(model,test_loader,base_loader,alg):
    
    interpretable_embedding = configure_interpretable_embedding_layer(model, "embeds")

    total_attr_cat, total_attr_num, X_ts_mask, data_ = [], [], [], []
    test_labels = []

    model.train()
    if alg == "GB":
        gb = GuidedBackprop(model)  
    elif alg == "IG":
        ig = IntegratedGradients(model)
    elif alg == "SVS":
        svs = ShapleyValueSampling(model)
        
    for (data,labels),(data_base,labels_base) in zip(test_loader,base_loader):
        nonzeros = ~data.cpu().numpy().any(axis=2)
        data = torch.Tensor(data).to(device)
        data_base = torch.Tensor(data_base).to(device)

        nonzero_index = np.argmax(nonzeros,axis=1)        
        nonzero_index = torch.LongTensor(nonzero_index)
        nonzero_index = torch.where(nonzero_index==0,torch.LongTensor([min_time]),nonzero_index)
        seq_lengths, perm_idx = nonzero_index.sort(0, descending=True)

        data = data[perm_idx]
        labels = labels[perm_idx]

        data_base = data_base[perm_idx]
        labels_base = labels_base[perm_idx]
      
        X_ts_bool = np.all(data.cpu().numpy()==0, axis = 2)
        X_ts_mask.append(X_ts_bool) 

        categorical = interpretable_embedding.indices_to_embeddings(data[:, :, 18:].long())
        numerical   = data[:, : ,:18]

        cat_base = interpretable_embedding.indices_to_embeddings(data_base[:, :, 18:].long())
        num_base   = data_base[:, : ,:18]

        if alg == "GB":
            (attr_num,attr_cat) = gb.attribute(inputs=(numerical,categorical), additional_forward_args= seq_lengths,target=1)
        elif alg == "IG":
            (attr_num,attr_cat) = ig.attribute(inputs=(numerical,categorical),baselines=(num_base,cat_base), additional_forward_args= seq_lengths,target=1)
        elif alg == "SVS":
            (attr_num,attr_cat) = svs.attribute(inputs=(numerical,categorical),baselines=(num_base,cat_base), additional_forward_args= seq_lengths,target=1)

    
        total_attr_cat.append(attr_cat.detach().cpu().numpy())
        total_attr_num.append(attr_num.detach().cpu().numpy())
        
        data_.append(data.detach().cpu().numpy())
    
    

    remove_interpretable_embedding_layer(model, interpretable_embedding)

    total_attr_cat = np.vstack(total_attr_cat)
    total_attr_cat = np.mean(total_attr_cat,axis=-1)

    total_attr_num = np.vstack(total_attr_num)

    X_ts_mask = np.vstack(X_ts_mask)

    total_attr_1 = np.concatenate([total_attr_num,total_attr_cat],axis=-1)
    total_attr_1[X_ts_mask] = np.nan

    feature_imp_1 = np.nanmean(total_attr_1, axis=1) #p 
    
    data_for_shap = np.vstack(data_)
    
    return feature_imp_1,data_for_shap


def get_ranking_cv_for_shap(X,y,alg="GB"):
    i = 1
    ranking = []
    data_= []
    
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED_VALUE)
    for train, test in kfold.split(X, y):
        print(f'Fold: {i}')
        X_tr = X[train]
        X_ts = X[test]
        y_tr = y[train]
        y_ts = y[test]

        X_base,y_base = get_baseline(X_ts,y_ts)
        ########################################################################
        params = {'lr':0.000075,'num_layers':1,'hidden_units':128,'n_classes':2,
              'dropout':0.3,'epochs':50,'input_size':33}

        LOADPATH = "eicu_%d_%d_fold_%d_cv_torch.pt"%(min_time,skip_time,i)

        BATCH_SIZE = 128
        ########################################################################
        print(LOADPATH)
        
        base_data    = overdata(X_base,y_base)
        test_data    = overdata(X_ts,y_ts)
        
        
        base_loader  = DataLoader(base_data, batch_size=BATCH_SIZE,shuffle=False,**kwargs)
        test_loader  = DataLoader(test_data, batch_size=BATCH_SIZE,shuffle=False,**kwargs)

        model = BiRNN(params['input_size'], params['hidden_units'], params['num_layers'], num_classes=params['n_classes'],drp = params['dropout']).to(device)
        model.load_state_dict(torch.load(LOADPATH))
        i += 1  
        
        if alg == "GB":
            feat_rank,data_for_shap = get_feature_ranking_for_shap(model,test_loader,base_loader,alg="GB")
        elif alg == "IG":
            feat_rank,data_for_shap = get_feature_ranking_for_shap(model,test_loader,base_loader,alg="IG")
        elif alg == "SVS":
            feat_rank,data_for_shap = get_feature_ranking_for_shap(model,test_loader,base_loader,alg="SVS")
            
        data_.append(data_for_shap)
        ranking.append(feat_rank)
    
    return ranking, data_

ranking_per_fold,data_per_fold = get_ranking_cv_for_shap(X_train,y_train,alg="GB")



In [21]:
import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 40}
matplotlib.rc('font', **font)

In [ ]:
def visualize_importances_dot(figname,feature_names, importances, alg, plot=True, axis_title="Features"):
    summary_plot(importances,features=np.nanmean(np.vstack(data_per_fold),axis=1) ,feature_names=feature_name, show=False,plot_type='dot', sort=True,max_display=21, use_log_scale=False)

    plt.tight_layout()
    if alg == "GB":
        plt.xlabel('Gradient Values')
    elif alg == "IG":
        plt.xlabel('Integrated Gradient')
    elif alg == "SVS":
        plt.xlabel('SHAP Values')

    plt.savefig(figname, dpi=100, bbox_inches='tight')
    plt.show()


if alg == "GB":
    savefigname = "{}min_{}skip_eicu_{}_jama".format(min_time,skip_time,alg)
    visualize_importances_dot(savefigname,feature_name, np.vstack(ranking_per_fold),alg="GB")    
elif alg == "IG":
    savefigname = "{}min_{}skip_eicu_{}_jama".format(min_time,skip_time,alg)
    visualize_importances_dot(savefigname,feature_name, np.vstack(ranking_per_fold),alg="IG")
elif alg == "SVS":
    savefigname = "{}min_{}skip_eicu_{}_jama".format(min_time,skip_time,alg)
    visualize_importances_dot(savefigname,feature_name, np.vstack(ranking_per_fold),alg="SVS")

# MIMIC

## Train 

In [32]:
def get_mimic_feature(min_time,skip_time):
    
    from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
    import numpy as np
    import re
    import os
    import csv
    import sys
    import pandas as pd
    import numpy as np
    import random
    from scipy import interp
    import matplotlib.pyplot as plt

    from itertools import cycle
    from sklearn.metrics import roc_curve, auc
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split, StratifiedKFold

    from sklearn.metrics import classification_report,confusion_matrix,f1_score,auc,roc_curve,precision_recall_curve
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc,confusion_matrix, average_precision_score, matthews_corrcoef
    from numpy.random import seed
    from inspect import signature

    import keras
    from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
    import warnings
    warnings.filterwarnings('ignore')

    from sklearn.linear_model import LogisticRegression
    from numpy.random import seed


    import torch.nn as nn
    import optuna
    from torch.utils.data.sampler import WeightedRandomSampler
    import torch
    from torch.utils.data import DataLoader,Dataset
    import torch 
    import torch.nn as nn
    from torch.nn.utils.rnn import pack_sequence

    SEED_VALUE = 36
    seed(SEED_VALUE)
    np.random.seed(SEED_VALUE)
    random.seed(SEED_VALUE)
    os.environ['PYTHONHASHSEED']=str(SEED_VALUE)



    torch.manual_seed(SEED_VALUE)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(SEED_VALUE)
    
    CV = True
    BATCH_SIZE = 128
    TASK = '%d_%d'%(min_time,skip_time)
    
    class BiRNN(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, num_classes, drp):
            super(BiRNN, self).__init__()
            self.num_layers  = num_layers
            self.num_classes = num_classes
            self.drop = nn.Dropout(p = drp)
            self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True)
            self.fc = nn.Linear(hidden_size*2, num_classes) # 2 for bidirection
            self.sigmoid = torch.nn.Sigmoid()
            self.embeds = nn.Embedding(50,5)
            self.hidden_size= hidden_size


        def forward(self, x_num, x_cat , seq_lengths):
            embed = self.embeds(x_cat).reshape((x_cat.size(0), x_cat.size(1), -1))
            x = torch.cat((x_num,embed),-1)
            out, _ =  self.lstm(x) # tensor of shape (batch_size, seq_length, hidden_size*2)
            lstm_out = out[:,-1,:]
            output = self.fc(lstm_out)#torch.cat((lstm_out_fw, lstm_out_bw), -1))
            return output
    
###########################################################
    def plot_cum_auc(x_y, models, savename='plot.png', x_label='False Positive Rate', y_label='True Positive Rate',auprc=False, thr=0):
    
        colors = ['b', 'r','g']
        for i, (x_y_tuple, m) in enumerate(zip(x_y, models)):
            plt.plot(x_y_tuple[0], x_y_tuple[1],label='{}: {:.4f}'.format(m,auc(x_y_tuple[0], x_y_tuple[1])), color=colors[i])
        if auprc:
            plt.plot([0, 1], [thr, thr], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(thr), alpha=.8)

        else:
            plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='k',label="Random: {:.4f}".format(0.5000), alpha=.8)

        plt.xlabel(x_label, fontsize=15)
        plt.ylabel(y_label, fontsize=15)
        plt.axhline(0, color='black')
        plt.axvline(0, color='black')

        if auprc:
            legend = plt.legend(loc="upper left", prop={'size': 10}, bbox_to_anchor=(0.6, 0.95))
        else:
            legend = plt.legend(loc="lower right", prop={'size': 10}, bbox_to_anchor=(1, 0.05))
            
        plt.savefig(savename,
                     dpi=400, facecolor='white', transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')
        plt.show()
        
        
    def compute_metrics(X_test,y_test,y_probs):

        fpr, tpr, thresholds = roc_curve(y_test, y_probs)
        specat90 = 1-fpr[tpr>=0.90][0]
        intrp = interp(np.linspace(0, 1, 100), fpr, tpr)
        intrp[0] = 0.0

        roc_auc = auc(fpr, tpr)

        TN,FP,FN,TP = confusion_matrix(y_test, y_probs.round()).ravel()
        PPV = TP/(TP+FP)
        NPV = TN/(TN+FN)


        mcc = matthews_corrcoef(y_test, y_probs.round())
        
        average_precision = average_precision_score(y_test, y_probs)
        precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
        prs = interp(np.linspace(0, 1, 100), recall[::-1], precision[::-1])
        prs[0] = 1.0
        prs[-1] = 0.0

        auc_prc = auc(recall, precision)

        return {'specat90': specat90, 'intrp': intrp,
                'fpr': fpr,
                'tpr': tpr, 'auc': roc_auc,
                'ppv': PPV, 'npv': NPV,
                'auc_prc': auc_prc,
                'prc':precision,
                'prs':prs,
                'rec':recall,
                'mcc': mcc}


    def avg_metrics(results):
    
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0, 1, 100)

        mean_tpr = np.mean([results[k]['intrp'] for k in results], axis=0)
        mean_tpr[-1] = 1.0
        std_tpr = np.std([results[k]['intrp'] for k in results], axis=0)
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std([results[k]['auc'] for k in results])
        ppvs = np.mean([results[k]['ppv'] for k in results])
        npvs = np.mean([results[k]['npv'] for k in results])
        mccs = np.mean([results[k]['mcc'] for k in results])
        specat90 = np.mean([results[k]['specat90'] for k in results])

        mean_precision = np.mean([results[k]['prs'] for k in results], axis=0)
        mean_precision[-1] = 0.0
        #########################
        mean_precision[0] = 1.0
        ########################
        mean_auc_prc = auc(mean_recall, mean_precision)
        std_auc_prc = np.std([results[k]['auc_prc'] for k in results], axis=0)

        print("Mean AUC: {0:0.4f} +- STD{1:0.4f}".format(mean_auc,std_auc))
        print("PPV: {0:0.4f}".format(ppvs))
        print("NPV: {0:0.4f}".format(npvs))
        print("MCC: {0:0.4f}".format(mccs))
        print("Spec@90: {0:0.4f}".format(specat90))

        return {'mean_auc': mean_auc,
                'tpr': mean_tpr,
                'std_auc':std_auc,
                'std_tpr': std_tpr,
                'ppv':ppvs,
                'npv':npvs,
                'mean_auc_prc':mean_auc_prc,
                'std_auc_prc':std_auc_prc,
                'mean_prc':mean_precision,
                'mean_recall':mean_recall,
                'mcc':mccs,
                'spec@90':specat90}

    def plot_auc(result):
        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='red',
                 label='Random', alpha=.8)

        mean_fpr = np.linspace(0,1,100)
        mean_tpr = result['tpr']
        mean_auc = result['mean_auc']
        std_auc = result['std_auc']
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.4f $\pm$ %0.4f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = result['std_tpr']
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()
###########################################################################
    
    #pos
    def pos_selection(df_pos,skip_time,min_time):
        posl = []
        posdf = pd.DataFrame(columns=df_pos.columns)
        all_matches = df_pos[df_pos['labelpt'] == df_pos['labelrec']]

        for i, p_id in enumerate(all_matches['patientunitstayid'].unique()):
            df_p_id = df_pos[df_pos['patientunitstayid'] == p_id]
            idx = all_matches[all_matches['patientunitstayid'] == p_id].index[0]
            t = df_pos.iloc[idx].itemoffset
            if t > (min_time + skip_time):
                posl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) & (df_p_id['itemoffset'] > (t-(skip_time+min_time)))])

        posdf = pd.concat(posl,axis=0)
        return posdf

    #neg
    def neg_selection(df_neg,skip_time,min_time):
        negl = []
        negdf = pd.DataFrame(columns=df_neg.columns)

        for i, p_id in enumerate(df_neg['patientunitstayid'].unique()):
            df_p_id = df_neg[df_neg['patientunitstayid'] == p_id]
            t = df_p_id.iloc[-1].itemoffset
            if t > (min_time + skip_time):
                negl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) &(df_p_id['itemoffset'] > (t-(min_time+skip_time)))])
        negdf = pd.concat(negl,axis=0)
        return negdf
#     eicu_df  = pd.read_csv("eicu_df_all_24los_normed.csv")
    mimic_df = pd.read_csv("mimic_df_all_24los_normed.csv")

 # ADD VASO
    mimic_df['vaso_dose'] = np.nan
    mimic_df['vaso_dose'] = mimic_df['rate_epinephrine']+mimic_df['rate_norepinephrine'] + mimic_df['rate_phenylephrine']/10 + mimic_df['rate_dopamine']/2
    mimic_df.drop(columns=['rate_epinephrine', 'rate_norepinephrine','rate_phenylephrine','rate_dopamine'],inplace=True)

#     import pdb;pdb.set_trace()

    mimic_pos = mimic_df[mimic_df['CAM']==1]
    mimic_neg = mimic_df[mimic_df['CAM']==0]
    
#     eicu_pos_filtered = pos_selection(eicu_pos,skip_time,min_time)
#     eicu_neg_filtered = neg_selection(eicu_neg,skip_time,min_time)
#     eicu_df_filtered = pd.concat([eicu_pos_filtered, eicu_neg_filtered],axis=0)

    mimic_pos_filtered = pos_selection(mimic_pos,skip_time,min_time)
    mimic_neg_filtered = neg_selection(mimic_neg,skip_time,min_time)
    mimic_df_filtered = pd.concat([mimic_pos_filtered, mimic_neg_filtered],axis=0)

#     trg = eicu_df_filtered.groupby('patientunitstayid')
    tsg  = mimic_df_filtered.groupby('patientunitstayid')

    idts = []
    test_np = []
    for idx, frame in tsg:
        idts.append(idx)
        test_np.append(frame)

#     idtr = []
#     train_np = []
#     for idx, frame in trg:
#         idtr.append(idx)
#         train_np.append(frame)


    columns_ord = ['patientunitstayid', 'itemoffset', 'gender','sofa', 'sofa_wo_gcs', 'age', 'admissionheight',
           'admissionweight', 'Heart Rate', 'O2 Saturation', 'glucose',
           'Temperature (C)', 'sodium', 'BUN', 'WBC x 1000', 'Hemoglobin',
           'Platelets', 'Potassium', 'Chloride', 'Bicarbonate', 'Creatinine',
            'vent_flag', 'vaso_dose', 'CAM']

    def reader_deli(df_list,verbose=1):
        X_noncat = []
        X_cat = []
        deli = []
        nrows = []
        ts = []
        PID = []
        nb_unit_stays = len(df_list)
        for i, df in enumerate(df_list):
            if verbose:
                sys.stdout.write('\rFeed StayID {0} of {1}...'.format(i+1, nb_unit_stays))
            dft = df
            dummy = pd.DataFrame(columns=columns_ord)
            for c in columns_ord:
                dummy[c] = dft[c]        
            dft = dummy
            narr = np.array(dft)
            pid = narr[0,0]
            x_cat    = narr[:,2:5]
            x_noncat = narr[:, 5:-1]
            labeldeli = narr[0, -1]
            time = narr[:,1]
            X_cat.append(x_cat)
            X_noncat.append(x_noncat)
            deli.append(labeldeli)
            ts.append(time)
            nrows.append(narr.shape[0])
            PID.append(pid)
        PID = np.array(PID)    
        X_cat = np.array(X_cat)
        X_noncat = np.array(X_noncat)
        deli = np.array(deli)
        ts= np.array(ts)
        return PID,X_cat,X_noncat,ts,nrows,deli
    
#     PID_tr,X_cat_tr,X_num_tr,ts_tr,nrows_tr,y_tr = reader_deli(train_np)
    PID_ts,X_cat_ts,X_num_ts,ts_ts,nrows_ts,y_ts = reader_deli(test_np)
    
    def pad(arr, max_len= min_time):
        tmp = np.zeros((max_len, arr.shape[1]))
        tmp[:arr.shape[0], :arr.shape[1]] = arr
        return tmp  
    def ret_numpy(X):
        X_list = []
        for n in X:
            X_list.append(pad(n))
        return np.array(X_list)
    
#     X_cat_tr = ret_numpy(X_cat_tr)
#     X_num_tr = ret_numpy(X_num_tr)

    X_cat_ts = ret_numpy(X_cat_ts)
    X_num_ts = ret_numpy(X_num_ts)
    
#     X_train = np.concatenate([X_num_tr,X_cat_tr],axis=-1)
#     y_train = y_tr
    X_test  = np.concatenate([X_num_ts,X_cat_ts],axis=-1)
    y_test  = y_ts

    
    def data_reader(X,y):
        return X,y

    
    class overdata(Dataset):
        def __init__(self, data,label):
            self.data  = torch.FloatTensor(data.astype('float'))
            self.label = torch.FloatTensor(label.astype('float'))

        def __len__(self):
            return len(self.data)

        def __getitem__(self, index):
            label  = self.label[index]
            data   = self.data[index]
            return data,label

    device = "cuda" if torch.cuda.is_available() else "cpu"
    kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}

    def sampler_fn(data, label):
        train_class_sample_count = torch.tensor([(torch.tensor(label) == t).sum() for t in torch.unique(torch.tensor(label), sorted=True)])
        train_weight = 1 / train_class_sample_count.float()
        train_samples_weight = torch.tensor([train_weight[i] for i in torch.tensor(label).long()])
        train_sampler = WeightedRandomSampler(train_samples_weight, len(train_samples_weight))
        return train_sampler
    
    def get_class_weights(data, label):
        neg = data[label==0].shape[0]
        pos = len(data) - neg
        weight_for_0 = (1 / neg)*((pos+neg))/2.0 
        weight_for_1 = (1 / pos)*((pos+neg))/2.0
        return weight_for_0 ,(weight_for_1)
    
    if CV:
        X_test,y_test= data_reader(X_test,y_test)
        
    sequence_length = min_time
    input_size = 33 
        
    def run_train(data_loader,params,data,label):
    
        train_losses, val_losses = [], []

        cw_0,cw_1 = get_class_weights(data,label)
        model = BiRNN(input_size, params['hidden_units'], params['num_layers'], num_classes=params['n_classes'],drp = params['dropout']).to(device)

        # Loss and optimizer
        criterion = nn.CrossEntropyLoss(weight=torch.Tensor([cw_0,cw_1]).to(device))
        optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'],weight_decay=1e-6)

        for epoch in range(params['epochs']):

            true_labels = []
            pred_labels = []

            running_loss = 0

            for data, labels in data_loader:
#                 import pdb;pdb.set_trace()
                model.train()
                data = torch.FloatTensor(data).to(device)
                labels = torch.FloatTensor(labels).to(device)
                labels = labels.long()

                ##
                nonzeros = ~data.cpu().numpy().any(axis=2)
                nonzero_index = np.argmax(nonzeros,axis=1)        
                nonzero_index = torch.LongTensor(nonzero_index)
    #             import pdb;pdb.set_trace()
                nonzero_index = torch.where(nonzero_index==0,torch.LongTensor([min_time]),nonzero_index)
                seq_lengths, perm_idx = nonzero_index.sort(0, descending=True)

                data = data[perm_idx]
                labels = labels[perm_idx]
                ###
                data = data.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(data[:, :, :18], data[:, : ,18:].long(), seq_lengths)
                loss = criterion(outputs, labels)

                # Backward and optimize
                loss.backward()
                optimizer.step()        
                running_loss += loss.item()
            print("Epoch: {}/{} ".format(epoch+1, params['epochs']),"Training Loss: {:.3f} ".format(running_loss/len(data_loader)))
        
        return model

    params = {'lr':0.000075,'num_layers':1,'hidden_units':128,'n_classes':2,
              'dropout':0.3,'epochs':50}
    
    def run_test(data_loader,model):
        with torch.no_grad():
            correct = 0
            total = 0
            model.eval()
            true_labels, pred_labels = [], []

            for data,labels in data_loader:
                labels = labels.to(device)
                labels = labels.long()
                ###
                nonzeros = ~data.cpu().numpy().any(axis=2)
                data = torch.Tensor(data).to(device)
                nonzero_index = np.argmax(nonzeros,axis=1)        
                nonzero_index = torch.LongTensor(nonzero_index)
                nonzero_index = torch.where(nonzero_index==0,torch.LongTensor([min_time]),nonzero_index)

                seq_lengths, perm_idx = nonzero_index.sort(0, descending=True)
                data = data[perm_idx]
                labels = labels[perm_idx]
                ###
                outputs = nn.Softmax()(model(data[:, :, :18], data[:, : ,18:].long(), seq_lengths))
                pred = outputs.detach().cpu().numpy()
                pred = pred[:,1]
                true_labels.append(labels)
                pred_labels.append(pred)
            true_labels = [l.item() for labels in true_labels for l in labels]
            pred_labels = [p for labels in pred_labels for p in labels]
            pred_labels = np.array(pred_labels)

            return true_labels,pred_labels

    def model_cv(X,y):
        i = 1
        cv_scores = {}
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED_VALUE)
        for train, test in kfold.split(X, y):
            print(f'Fold: {i}')
            X_tr = X[train]
            X_ts = X[test]
            y_tr = y[train]
            y_ts = y[test]

            data_sampler = sampler_fn(X_tr,y_tr)
            train_data = overdata(X_tr,y_tr)
            test_data  = overdata(X_ts,y_ts)
            train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=False,sampler=data_sampler, **kwargs)
            test_loader  = DataLoader(test_data, batch_size=BATCH_SIZE,shuffle=False, **kwargs)

            model = run_train(train_loader, params, X_tr, y_tr)
            savepath = "mimic_%d_%d_fold_%d_cv_torch.pt"%(min_time,skip_time,i)
            
            torch.save(model.state_dict(), savepath)
            print(f"Model:{savepath} saved.")

            y_ts,probs = run_test(test_loader,model)
            cv_scores[i] = compute_metrics(X_ts,y_ts,probs)
            i += 1        
        cum_scores = avg_metrics(cv_scores)
#         plot_cum_auc(cum_scores)
        return cum_scores
    


    
    models = ['LSTM']
    # models = ['LSTM']
    results = {}
    for m in models:
        print(f'**********Model: {m} *********')
        results[m] = model_cv(X_test,y_test)
        mean_fpr = np.linspace(0,1,100)
        fpr_tprs = [(mean_fpr, results[m]['tpr']) for m in models]
        plot_cum_auc(fpr_tprs, models, savename='{}_cv_mimic_feat.png'.format(TASK),auprc=False)

   

In [33]:
for m in [12]:
    for s in [48]:
        get_mimic_feature(m,s)

# Ranking

## Read data for ranking

In [45]:
import pandas as pd
import numpy as np
import sys


min_time = 24
skip_time = 96


device = "cuda" if torch.cuda.is_available() else "cpu"
kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}


#pos
def pos_selection(df_pos,skip_time,min_time):
    posl = []
    posdf = pd.DataFrame(columns=df_pos.columns)
    all_matches = df_pos[df_pos['labelpt'] == df_pos['labelrec']]

    for i, p_id in enumerate(all_matches['patientunitstayid'].unique()):
        df_p_id = df_pos[df_pos['patientunitstayid'] == p_id]
        idx = all_matches[all_matches['patientunitstayid'] == p_id].index[0]
        t = df_pos.iloc[idx].itemoffset
        if t > (min_time + skip_time):
            posl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) & (df_p_id['itemoffset'] > (t-(skip_time+min_time)))])

    posdf = pd.concat(posl,axis=0)
    return posdf

#neg
def neg_selection(df_neg,skip_time,min_time):
    negl = []
    negdf = pd.DataFrame(columns=df_neg.columns)

    for i, p_id in enumerate(df_neg['patientunitstayid'].unique()):
        df_p_id = df_neg[df_neg['patientunitstayid'] == p_id]
        t = df_p_id.iloc[-1].itemoffset
        if t > (min_time + skip_time):
            negl.append(df_p_id[(df_p_id['itemoffset'] < (t-skip_time)) &(df_p_id['itemoffset'] > (t-(min_time+skip_time)))])
    negdf = pd.concat(negl,axis=0)
    return negdf
mimic_df = pd.read_csv("mimic_df_all_24los_normed.csv")

# ADD VASO
mimic_df['vaso_dose'] = np.nan
mimic_df['vaso_dose'] = mimic_df['rate_epinephrine']+mimic_df['rate_norepinephrine'] + mimic_df['rate_phenylephrine']/10 + mimic_df['rate_dopamine']/2
mimic_df.drop(columns=['rate_epinephrine', 'rate_norepinephrine','rate_phenylephrine','rate_dopamine'],inplace=True)

mimic_pos = mimic_df[mimic_df['CAM']==1]
mimic_neg = mimic_df[mimic_df['CAM']==0]


mimic_pos_filtered = pos_selection(mimic_pos,skip_time,min_time)
mimic_neg_filtered = neg_selection(mimic_neg,skip_time,min_time)
mimic_df_filtered = pd.concat([mimic_pos_filtered, mimic_neg_filtered],axis=0)

tsg  = mimic_df_filtered.groupby('patientunitstayid')

idts = []
test_np = []
for idx, frame in tsg:
    idts.append(idx)
    test_np.append(frame)



columns_ord = ['patientunitstayid', 'itemoffset', 'gender','sofa', 'sofa_wo_gcs', 'age', 'admissionheight',
       'admissionweight', 'Heart Rate', 'O2 Saturation', 'glucose',
       'Temperature (C)', 'sodium', 'BUN', 'WBC x 1000', 'Hemoglobin',
       'Platelets', 'Potassium', 'Chloride', 'Bicarbonate', 'Creatinine',
        'vent_flag', 'vaso_dose', 'CAM']

def reader_deli(df_list,verbose=1):
    X_noncat = []
    X_cat = []
    deli = []
    nrows = []
    ts = []
    PID = []
    nb_unit_stays = len(df_list)
    for i, df in enumerate(df_list):
        if verbose:
            sys.stdout.write('\rFeed StayID {0} of {1}...'.format(i+1, nb_unit_stays))
        dft = df
        dummy = pd.DataFrame(columns=columns_ord)
        for c in columns_ord:
            dummy[c] = dft[c]        
        dft = dummy
        narr = np.array(dft)
        pid = narr[0,0]
        x_cat    = narr[:,2:5]
        x_noncat = narr[:, 5:-1]
        labeldeli = narr[0, -1]
        time = narr[:,1]
        X_cat.append(x_cat)
        X_noncat.append(x_noncat)
        deli.append(labeldeli)
        ts.append(time)
        nrows.append(narr.shape[0])
        PID.append(pid)
    PID = np.array(PID)    
    X_cat = np.array(X_cat)
    X_noncat = np.array(X_noncat)
    deli = np.array(deli)
    ts= np.array(ts)
    return PID,X_cat,X_noncat,ts,nrows,deli

PID_ts,X_cat_ts,X_num_ts,ts_ts,nrows_ts,y_ts = reader_deli(test_np)

def pad(arr, max_len= min_time):
    tmp = np.zeros((max_len, arr.shape[1]))
    tmp[:arr.shape[0], :arr.shape[1]] = arr
    return tmp  
def ret_numpy(X):
    X_list = []
    for n in X:
        X_list.append(pad(n))
    return np.array(X_list)


X_cat_ts = ret_numpy(X_cat_ts)
X_num_ts = ret_numpy(X_num_ts)

X_test  = np.concatenate([X_num_ts,X_cat_ts],axis=-1)
y_test  = y_ts

Feed StayID 773 of 773...

# Interpretable models

In [ ]:
from shap import summary_plot

def get_feature_ranking_for_shap(model,test_loader,base_loader):

    interpretable_embedding = configure_interpretable_embedding_layer(model, "embeds")

    total_attr_cat, total_attr_num, X_ts_mask, data_ = [], [], [], []
    test_labels = []

    model.train()
    
    ig = IntegratedGradients(model)
    svs = ShapleyValueSampling(model)
    gb = GuidedBackprop(model)
    
    for (data,labels),(data_base,labels_base) in zip(test_loader,base_loader):
        nonzeros = ~data.cpu().numpy().any(axis=2)
        data = torch.Tensor(data).to(device)
        data_base = torch.Tensor(data_base).to(device)

        nonzero_index = np.argmax(nonzeros,axis=1)        
        nonzero_index = torch.LongTensor(nonzero_index)
        nonzero_index = torch.where(nonzero_index==0,torch.LongTensor([min_time]),nonzero_index)
        seq_lengths, perm_idx = nonzero_index.sort(0, descending=True)

        data = data[perm_idx]
        labels = labels[perm_idx]

        data_base = data_base[perm_idx]
        labels_base = labels_base[perm_idx]
      
        X_ts_bool = np.all(data.cpu().numpy()==0, axis = 2)
        X_ts_mask.append(X_ts_bool) 

        categorical = interpretable_embedding.indices_to_embeddings(data[:, :, 18:].long())
        numerical   = data[:, : ,:18]

        cat_base = interpretable_embedding.indices_to_embeddings(data_base[:, :, 18:].long())
        num_base   = data_base[:, : ,:18]

        (attr_num,attr_cat) = ig.attribute(inputs=(numerical,categorical),baselines=(num_base,cat_base),additional_forward_args= seq_lengths,target=1)
        (attr_num,attr_cat) = svs.attribute(inputs=(numerical,categorical),baselines=(num_base,cat_base),additional_forward_args= seq_lengths,target=1)
        (attr_num,attr_cat) = gb.attribute(inputs=(numerical,categorical), additional_forward_args= seq_lengths,target=1)
        total_attr_cat.append(attr_cat.detach().cpu().numpy())
        total_attr_num.append(attr_num.detach().cpu().numpy())
        data_.append(data.detach().cpu().numpy())


    remove_interpretable_embedding_layer(model, interpretable_embedding)

    total_attr_cat = np.vstack(total_attr_cat)
    total_attr_cat = np.mean(total_attr_cat,axis=-1)

    total_attr_num = np.vstack(total_attr_num)

    X_ts_mask = np.vstack(X_ts_mask)

    total_attr_1 = np.concatenate([total_attr_num,total_attr_cat],axis=-1)
    total_attr_1[X_ts_mask] = np.nan

    feature_imp_1 = np.nanmean(total_attr_1, axis=1) #p
    
    data_for_shap = np.vstack(data_)
    
    return feature_imp_1,data_for_shap



def get_ranking_cv_for_shap(X,y):
    i = 1
    ranking = []
    data_ = []
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED_VALUE)
    for train, test in kfold.split(X, y):
        print(f'Fold: {i}')
        X_tr = X[train]
        X_ts = X[test]
        y_tr = y[train]
        y_ts = y[test]

        X_base,y_base = get_baseline(X_ts,y_ts)
        ########################################################################
        params = {'lr':0.000075,'num_layers':1,'hidden_units':128,'n_classes':2,
              'dropout':0.3,'epochs':50,'input_size':33}

#         LOADPATH = "mimic_%s_%d_cv_torch.pt"%(24,96)
        LOADPATH = "mimic_%d_%d_fold_%d_cv_torch.pt"%(min_time,skip_time,i)

        BATCH_SIZE = 128
        ########################################################################
        print(LOADPATH)
        
        base_data    = overdata(X_base,y_base)
        test_data    = overdata(X_ts,y_ts)
        
        
        base_loader  = DataLoader(base_data, batch_size=BATCH_SIZE,shuffle=False,**kwargs)
        test_loader  = DataLoader(test_data, batch_size=BATCH_SIZE,shuffle=False,**kwargs)

        model = BiRNN(params['input_size'], params['hidden_units'], params['num_layers'], num_classes=params['n_classes'],drp = params['dropout']).to(device)
        model.load_state_dict(torch.load(LOADPATH))
        i += 1  
        
        feat_rank,data_for_shap = get_feature_ranking_for_shap(model,test_loader,base_loader)

    
        data_.append(data_for_shap)
        ranking.append(feat_rank)
    
    return ranking, data_

ranking_per_fold_mimic,data_per_fold_mimic = get_ranking_cv_for_shap(X_test,y_test)


def visualize_importances_dot(figname,feature_names, importances, plot=True, axis_title="Features"):
    summary_plot(importances,features=np.nanmean(np.vstack(data_per_fold_mimic),axis=1),max_display=21 ,feature_names=feature_name, show=False,plot_type='dot', sort=True, use_log_scale=False)
        
    plt.tight_layout()
    plt.xlabel('Integrated Gradient')
    plt.xlabel('SHAP Values')
    plt.xlabel('Gradient Values')
    
    plt.savefig(figname, dpi=450)
    plt.show()

savefigname = "{}min_{}skip_mimic_GB_jama".format(min_time,skip_time)

visualize_importances_dot(savefigname,feature_name, np.vstack(ranking_per_fold_mimic))